# ***Installation Requirements***



In [6]:
!pip install tensorflow==1.14

# ***Mount Google Drive***

In [7]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/TES-RNN-GPU_Github/Capacity_Forecasting

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/TES-RNN-GPU_Github/Capacity_Forecasting


# ***Imports***

In [8]:
import math
import numpy as np
from torch.utils.data import DataLoader
from data_loading import create_dataset, Dataset
from config import get_config
from trainer import TESRNNTrainer
from validator import TESRNNValidator
from tester import TESRNNTester
from model import TESRNN
from loss_modules import *

# ***TES-RNN***

In [9]:
# CONFIGURATION SETTINGS

# List of the services to be tested
#services = ['Facebook', 'Instagram', 'Snapchat', 'Twitter', 'YouTube']
services = ['Facebook']

# Number of clusters
num_clusters = 1

# List of alphas to be tested
alphas = [5]

# Define the number of training epochs
epochs = 1

# Define the number of training batch size
batch_size = 288

# Define the number of train, validation and test samples
train_samples = 16128
val_samples = 4032
test_samples = 2016

# Define the input size and output size of the prediction
input_size = 6
output_size = 1

# Golden ratio for the golden search algorithm
gratio = (math.sqrt(5) + 1) / 2

# Stopping condition value for the golden search algorithm (interval length)
stop_value = 0.01

In [10]:
# SIMULATION RUNS
num_runs = 1

# Simulations over different services
for service in services:

    # Simulations over different alpha values
    for alpha in alphas:

        # Configuration loading
        config = get_config('Traffic', epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_size, output_size)
    
        # Data loading
        data = '../../Dataset/' + service + '/time_load_cor_matrix.npy'
        train, val, test = create_dataset(data, config['chop_train'], config['chop_val'], config['chop_test'])
        dataset = Dataset(train, val, test, config['device'])
    
        # Maximum of single cluster traffic in the training set (for normalization)
        maximum = np.max(train[0])
    

        # Running many simulations for a given service and alpha
        for i in range(1, num_runs+1):

            # Initial extremes of the interval of the Minimum Level Threshold tau (expressed as fraction of maximum)
            tau_min = 0.0
            tau_max = 1.0
    
            # Current extremes of the interval of tau
            c = tau_min
            d = tau_max
    
            # Iterations counter for golden search algorithm
            iterations = 1
    
            # Dictionary collecting denormalized validation loss values for a given tau
            val_dict = {}
    

            # Stopping condition for golden search algorithm
            while abs(tau_max - tau_min) > stop_value:
        
                # Determine current Minimum Level Threshold tau
                if (iterations%3) > 0:
                    # Try tau as left extreme    
                    if (iterations%3) == 1:
                        tau = c
                    # Try tau as right extreme
                    else:
                        tau = d
        

                # Run actual golden search algorithm 
                else:

                    # Determine the new extreme of tau interval
                    if f_c < f_d:
                        print("\nNew right-extreme of the interval is %f" % d)
                        tau_max = d
                    else:
                        print("\nNew left-extreme of the interval is %f" % c)
                        tau_min = c
                
                    print("Current length of tau interval is %f \n" % abs(tau_max - tau_min))
                    c = tau_max - (tau_max - tau_min) / gratio
                    d = tau_min + (tau_max - tau_min) / gratio
                    iterations = iterations + 1
                    continue
        

                # Compute denormalized validation loss for current tau
                f_val = val_dict.get(round(tau,6))
                print("\nSearching a threshold in the interval [%f,%f]" % (tau_min, tau_max))
                print("Threshold for this run is %f" % tau)
        

                # Denormalized validation loss not yet calculated for current tau
                if f_val == None:
        
                    # Dataloader initialization
                    dataloader = DataLoader(dataset, batch_size=config['series_batch'], shuffle=False)

                    # Model initialization
                    run_id = service + '/Alpha_' + str(alpha) + '/Simulation_' + str(i)
                    model = TESRNN(tau = tau, maximum = maximum, num_clusters = num_clusters, config = config, run_id = run_id)

                    # Run model trainer
                    trainer = TESRNNTrainer(model, dataloader, run_id, config)
                    trainer.train_epochs()
    
                    # Run model validator
                    validator = TESRNNValidator(model, dataloader, run_id, config)
                    validator.validating()
        
                    # Compute denormalized validation loss
                    norm_preds = np.load('Results/' + run_id + '/val_predictions.npy')
                    norm_actuals = np.load('Results/' + run_id + '/val_actuals.npy')
                    levels = np.load('Results/' + run_id + '/val_levels.npy')
                    val_loss = denorm_validation_loss(norm_preds, norm_actuals, levels, alpha)
                    print("Denormalized validation loss for this run %f" % val_loss)
                    val_dict[round(tau,6)] = val_loss

                    # Set denormalized validation loss for interval extreme
                    if (iterations%3) == 1:
                        f_c = val_loss
                    else:
                        f_d = val_loss
        

                # Denormalized validation loss already calculated for current tau
                else:
                    print("Denormalized validation loss for this run %f" % f_val)
                    # Set denormalized validation loss for interval extreme
                    if (iterations%3) == 1:
                        f_c = f_val
                    else:
                        f_d = f_val
            

                # Increase algorithm iterations
                iterations = iterations + 1

        
    
            # Get the final optimal Minimum Level Threshold tau
            tau = (tau_min + tau_max) / 2
            print('\nFinally chosen threshold = %f\n' % tau)
            np.save('Results/' + run_id + '/optimal_tau.npy', tau)
    


            # Run the optimized model
    
            # Dataloader initialization
            dataloader = DataLoader(dataset, batch_size=config['series_batch'], shuffle=False)
    
            # Model initialization
            model = TESRNN(tau = tau, maximum = maximum, num_clusters = num_clusters, config = config, run_id = run_id)
    
            # Run model trainer
            trainer = TESRNNTrainer(model, dataloader, run_id, config)
            trainer.train_epochs()
    
            # Run model tester
            tester = TESRNNTester(model, dataloader, run_id, config)
            tester.testing()


Searching a threshold in the interval [0.000000,1.000000]
Threshold for this run is 0.000000


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Denormalized validation loss for this run 15666260865819.011719

Searching a threshold in the interval [0.000000,1.000000]
Threshold for this run is 1.000000
Denormalized validation loss for this run 76495348576060.156250

New right-extreme of the interval is 1.000000
Current length of tau interval is 1.000000 


Searching a threshold in the interval [0.000000,1.000000]
Threshold for this run is 0.381966
Denormalized validation loss for this run 20417304004235.574219

Searching a threshold in the interval [0.000000,1.000000]
Threshold for this run is 0.618034
Denormalized validation loss for this run 29549925739494.519531

New right-extreme of the interval is 0.618034
Current length of tau interval is 0.618034 


Searching a threshold in the interval [0.000000,0.618034]
Threshold for this run is 0.236068
Denormalized validation loss for this run 17089079405600.777344

Searching a threshold in the interval [0.000000,0.618034]
Threshold for this run is 0.381966
Denormalized validation lo